In [17]:
import torch
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch import nn 
from torch.utils.data import DataLoader
from torch import optim

train_data = CIFAR10(root='data', train=True, download=True, transform=transforms.ToTensor())

Files already downloaded and verified


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [53]:
class Cifar10_clf(nn.Module):
    def __init__(self):
        super(Cifar10_clf, self).__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(64 * 28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        
    def forward(self ,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    
model = Cifar10_clf().to(device)

In [54]:
loss_fn = nn.CrossEntropyLoss()
optim_fn = optim.AdamW(model.parameters(), 0.001)

In [55]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
def training_loop(epochs):
    model.train()
    running_loss = 0.0
    for epoch in range(epochs):
        
        for batch in train_loader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)

            # 1. Forward pass
            outputs = model(inputs)

            # 2. Compute loss
            loss = loss_fn(outputs, labels)

            # 3. Backward pass
            loss.backward()

            # 4. Optimizer step (update weights)
            optim_fn.step()

            # 5. Zero gradients for the next iteration
            optim_fn.zero_grad()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")


In [56]:
training_loop(1)

Epoch 1/1, Loss: 1.5415
